In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Flatten, Dropout
from keras.datasets import mnist
from keras.optimizers import Adam
import math
import numpy as np
import os
from PIL import Image

Using TensorFlow backend.


### 🐍mnistデータ読み込む🐍
- 読み込んだ後に画像の画素値を-1～1に正規化した

In [1]:
(X_train, _), (_, _) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

batch_size = 128

half_batch = int(batch_size / 2)

NameError: name 'keras' is not defined

### 🐍最適化関数はAdamに🐍
- なんでAdamにしたか？[ここ](https://www.renom.jp/ja/notebooks/tutorial/basic_algorithm/adam/notebook.html)参照してください。
- とても長いので集約すると、現時点では最も適切なパラメータ更新ができる最適化手法として認められているから。

In [3]:
z_dim = 100

img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

optimizer =Adam(0.0002, 0.5)

Instructions for updating:
Colocations handled automatically by placer.


### 🐍Generatorを設計🐍

In [4]:
noise_shape = (z_dim,)
generator = Sequential()

generator.add(Dense(256, input_shape=noise_shape))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(np.prod(img_shape), activation='tanh'))
generator.add(Reshape(img_shape))

### 🐍Discriminatorを設計🐍

In [5]:
img_shape = (img_rows, img_cols, channels)
discriminator = Sequential()

discriminator.add(Flatten(input_shape=img_shape))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='sigmoid'))

In [6]:
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

### 🐍combinedを設計🐍
- GeneratorとDiscriminatorをつなぎ合わせたものを作成しておく
- Generatorを学習させるときは、Generator単体ではなく、このCombinedを使って学習を行う

In [7]:
discriminator.trainable = False
combined = Sequential([generator, discriminator])
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

### 🐍学習🐍
- 規定回数ごとに画像を保存している
- ちょっとあとでもう少しわかりやすくします
- 最終的には、Discriminatorの正答率が50%になるくらいまで、Generatorに成長してもらいたい

In [8]:
for epoch in range(50000):
    
    # 以下、Discriminator学習
    # Generatorによる偽データの生成
    noise = np.random.normal(0, 1, (half_batch, z_dim))
    gen_imgs = generator.predict(noise)
    
    # バッチサイズの半数を教師データからピックアップ
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]
    
    # discriminator学習
    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    
    # それぞれの損失関数を平均
    d_loss_mean = np.add(d_loss_real, d_loss_fake) * 0.5
    
    # 以下、Generator学習
    noise = np.random.normal(0, 1, (batch_size, z_dim))
    g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
    
    if epoch % 100 == 0:
        gen_imgs = np.squeeze(gen_imgs)
        save_img = gen_imgs[0] * 127.5 + 127.5
        Image.fromarray(save_img.astype(np.uint8)).save('savefig/{}.png'.format(str(epoch)))
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss_mean[0], 100*d_loss_mean[1], g_loss))

Instructions for updating:
Use tf.cast instead.


C:\Users\Kanazashi\AppData\Local\conda\conda\envs\TestEnv\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.584364, acc.: 70.31%] [G loss: 0.775976]
100 [D loss: 0.321783, acc.: 89.84%] [G loss: 2.989811]
200 [D loss: 0.638144, acc.: 61.72%] [G loss: 1.432993]
300 [D loss: 0.670203, acc.: 46.09%] [G loss: 0.641549]
400 [D loss: 0.652354, acc.: 50.00%] [G loss: 0.680980]
500 [D loss: 0.651729, acc.: 57.03%] [G loss: 0.718492]
600 [D loss: 0.637874, acc.: 61.72%] [G loss: 0.760564]
700 [D loss: 0.631677, acc.: 66.41%] [G loss: 0.977111]
800 [D loss: 0.613783, acc.: 70.31%] [G loss: 0.847255]
900 [D loss: 0.598044, acc.: 71.88%] [G loss: 0.904730]
1000 [D loss: 0.565960, acc.: 80.47%] [G loss: 0.911945]
1100 [D loss: 0.579055, acc.: 77.34%] [G loss: 0.935300]
1200 [D loss: 0.594797, acc.: 71.09%] [G loss: 1.059432]
1300 [D loss: 0.607828, acc.: 64.06%] [G loss: 0.952157]
1400 [D loss: 0.566014, acc.: 70.31%] [G loss: 0.985570]
1500 [D loss: 0.576758, acc.: 71.09%] [G loss: 1.025192]
1600 [D loss: 0.622203, acc.: 61.72%] [G loss: 0.937348]
1700 [D loss: 0.588006, acc.: 73.44%] [G lo

### 🐍モデルの保存を行う
- generator, discriminator共に、モデルの構造自体を.jsonで保存し、重みは.h5ファイルで保存

In [18]:
generator_json_string = generator.to_json()

with open('saved_model/generator_model.json', 'w') as fw_g:
    fw_g.write(generator_json_string)

generator.save_weights('saved_model/generator_weights.h5')

discriminator_json_string = discriminator.to_json()

with open('saved_model/discriminator_model.json', 'w') as fw_d:
    fw_d.write(discriminator_json_string)

discriminator.save_weights('saved_model/discriminator_weights.h5')